## Project 1

Import the required libraries.

In [6]:

import numpy as np
from matplotlib import pyplot as plt
from iminuit import Minuit as im
from scipy import integrate
from scipy.stats import expon

# import warnings
# warnings.filterwarnings("error")


Establish the nominal values and other constants for the simulations. 

In [7]:
#nominal values
tau_ = 1.5
V_   = 0.1
dm_ = 20.0
s_= 0
error_s_ = 0.03

#detector time range
XMIN = 0 
XMAX = 20 #cutoff
#number of events
NEVENTS = int(1e+4)
NEVENTS_HIGH = int(1e+5)
#number of monte-carlo runs
NRUN = 500

#initializing the global time array of events
t = 0

Defining the functions that will be called.

The decay_toy function might be slighlty convulated so it is explained below:

The random decay times are generated by generating a specified size of random numbers between 0 and 1. This is scaled by the XMAX(x1) and then evaluated using the decay PDF(y1). Then another set of random numbers between 0 and 1 is generated and scaled by a number greather than fmax i.e the largest value the probability desnity function can generate(y2). We chose this to be 1. Indices where y2 is less than y1, correspond to the x1 values that are selected as the random events. While loop is used to generate more random numbers following the described conditions to fill the x1 array with random events where y2 initially was not less than y1. 

In the decay_toy function, it is separated by an if function, where the first part is catered for runs with only one value of 's' and second part is optimized for Part 3 to create random values for multiple runs with an array of 's' values. 

At the end, resolution error is introduced using the numpy random normal function that generates a random number from a guassian distribution with mwan = 0 and sigma = $f*tau\_$. When f = 0, there is no resolution error on the measurements. Sometimes, for a negative resolution error, the random value in x1 might become negative. However, this is rare and very small in value and does not cause a problem with the fitting. It is treated simply as a part of the error in the system. The negative values can also be disregarded but this also means replacing them suitable values to have N events. This slows it down and complicates the code unnecessarily  since it seems to still give reuslts within the systematic error.  

The function can do above in parallel for multiple monte-carlo experiments by utilizing the fact that numpy supports more than 1D manipulations.

In [16]:
def decay_pdf(tau=tau_,V=V_,dm=dm_,s=s_):
    """PDF function for the decay X -> D

    Args:
        tau (float, optional): lifetime parameter. Defaults to tau_.
        V (float, optional): Asymmetry parameter. Defaults to V_.
        dm (float, optional): mass difference parameter. Defaults to dm_.
        s (float, optional): time acceptance parameter. Defaults to s_.

    Returns:
        lambda function: returns a lambda function that can take input of a float or a numpy array
    """
    return lambda x: (1+V*np.sin(dm*x))*np.exp(-x/tau) * (1+x*s)
  


def decay_toy(N=1e+4,f=0.,s=s_,mc_runs=1,fmax=1):
    """Generates a array of random decay events that follows the decay pdf

    Args:
        N (int, optional): Total number of random events. Defaults to 1e+4.
        f (int, optional): time resolution fraction parameter. Defaults to 0.
        s (float/Numpy Array, optional): Time Acceptance Parameter. Defaults to s_.
        mc_runs (int, optional): Number of Monte Carlo runs. Defaults to 1.
        fmax (float, optional): fmax value for scaling random candidate set. Defaults to 1.


    Returns:
        Numpy Array: If m_runs is == 1, function returns an array of random events that follows the PDF.
        If m_runs is > 1, returns an array of experiments(runs) with each run containing an array of random events.
    """


    N = int(N)
    
    #Divide into these two statements to make it optimum speed for each condition. 
    # When s is an array of values for Part 3 and when its just a float or int. 
    
    if (type(s) == float) or (type(s) == int):
        evaluate_y = decay_pdf(tau_,V_,dm_,s)
        sampling  = np.random.uniform(size=(2,mc_runs,N))
        x1 = XMAX*sampling[0]
        y1 = evaluate_y(x1)
        y2 = fmax*sampling[1]
        left_filtered = ~np.less(y2,y1)

        while(np.any(left_filtered)):
            set_c = np.random.uniform(size=(2,np.count_nonzero(left_filtered)))
            x1[left_filtered] = XMAX*set_c[0]
            y1[left_filtered] = evaluate_y(x1[left_filtered])
            y2[left_filtered] =  fmax*set_c[1]
            
            left_filtered = ~np.less(y2,y1)
       
    else:
        evaluate_y = decay_pdf(tau_,V_,dm_,s[:, None])
  
        
        sampling  = np.random.uniform(size=(2,mc_runs,N))
        x1 = XMAX*sampling[0]
        y1 = evaluate_y(x1)
        y2 = fmax*sampling[1]
        left_filtered = ~np.less(y2,y1)

        while(np.any(left_filtered)):
            set_c = np.random.uniform(size=(2,np.count_nonzero(left_filtered)))
            x1[left_filtered] = XMAX*set_c[0]
            y1 = evaluate_y(x1) #this cannot be done by indexing when s is an array
            y2[left_filtered] =  fmax*set_c[1]
            
            left_filtered = ~np.less(y2,y1)
   
    x1 += np.random.normal(0,f*tau_,size=(mc_runs,N)) #adds time resolution error
   
    # uncomment below to disregard the negative events, but this also means < `N events. 
    # This can be fixed but it will also make the code slower and more complicated than 
    # needed for the estimations and the result is still within the systematic error.
    
    # x1 = x1[x1 >= 0]

    return x1

def nll(tau,V,dm,s=0):
    """NLL function of the decay PDF

    Args:
        tau (float): lifetime parameter
        V (float): asymmetry parameter 
        dm (float): mass difference parameter
        s (float, optional): time acceptance parameter. Defaults to 0.

    Returns:
        float : NLL of the decay PDF
    """
    
    global t
  
    # try:
    p = decay_pdf(tau,V,dm,s)
    # print(tau,V,dm)
    n = integrate.quad(p,XMIN,XMAX)
    P = p(t)/n[0]

    return -np.sum(np.log(P))
    # except:
    #     print(tau,V,dm)
        #    (1.4237337909006074, -0.31729950834892084 ,-141.7129126571873,0.03)
        #       (0.039125722396590665 -8.068114023694228 6.001290940604338)
        # Above some of the values for which the integration function was divergent or the Probability function was negative rendering error from the
        #logarithm.
        
def decay_fit(s=s_):
    """ Minimsation function for the NLL
    Args:
        s(float, optional): Time Acceptance Parameter. Defaults to s = s_
    Returns:
        iMinuit: Minimized object to obtain minimisation params
    """
    m = im(nll,tau=tau_, V=V_, dm=dm_,s=s)
    m.fixed['s'] = True
    m.limits['tau'] = [0,10]
    m.limits['V'] = [0,1]
    m.limits['dm'] = [0,100]
    m.errordef = 0.5
    m.migrad()

    return m
     

By making it such that warnings treated as errors and using try and except, it was observed that minuit would sometime input values that are too large or negative in any of the parameters leading to errors where the P - probability value is negative or the integrate in normalization is divergent, leading to warnings. This was avoided by setting the limit of the minuit to positive values and to remain within the order of magnitude of the nominal values. However, minuit might still use values that turn the integrand divergent but is now less likely within the limits.This does not affect the results in any significant way.

Example values that were given by minuit leading to warnings:
(1.4237337909006074 -0.31729950834892084 -141.7129126571873),

(0.039125722396590665 -8.068114023694228 6.001290940604338)

## Part 1

Monte-Carlo with each toy experiment with 100,000 events. 500 Monte-Carlo experiments were run.

500 was chosen since previous checkpoints finished in a acceptable amount of time for similar simulations.

**1.1 Monte-Carlo Dataset from 500 toy experiments.**

In [9]:
t_runs = decay_toy(1e+5,mc_runs=500)

**1.2 Minuit fit on each of the experiments and extracting the parameters estimation from each run.**

In [10]:
monte_carlo_data = np.empty((NRUN,3))
for i in range(NRUN):
    t = t_runs[i]
    monte_carlo_data[i] = decay_fit().values[0:3]

**1.3 Printing the results**

In [11]:
params = np.mean(monte_carlo_data,axis=0)
errors_std= np.std(monte_carlo_data,axis=0)
error_mean = errors_std/np.sqrt(NRUN)

print(f'Parameters Mean Value | Tau : {params[0]}, V: {params[1]}, dm: {params[2]}')
print(f'Standard Deviation | Tau : {errors_std[0]}, V: {errors_std[1]}, dm: {errors_std[2]}')
print(f'Error on the mean | Tau : {error_mean[0]}, V: {error_mean[1]}, dm: {error_mean[2]}')

Parameters Mean Value | Tau : 1.5033539105786968, V: 0.09675365317131081, dm: 19.999454805715473
Standard Deviation | Tau : 0.004780932954920731, V: 0.0043797450434185075, dm: 0.020490749023843555
Error on the mean | Tau : 0.00021380982166143382, V: 0.000195868152824031, dm: 0.0009163741545440329


**Note :** The statistical precision is the error on the mean obtained statistically from the Monte-Carlo.

For the run with 100,000 events, the parameter estimation from the monte-carlo experiment:

| Name  | Value | Error of the Mean(Statistical Precision) |  Standard Deviation
| ---   | ---   | ---   | --
| tau   | 1.5033  | 0.0002  |  0.005      
|  V    | 0.0969 | 0.0002  |   0.004
|  Δm    | 19.999  | 0.001 |  0.02


**1.4 Similar steps followed for now N = 1e+4**

In [12]:
t_runs = decay_toy(N = 1e+4,mc_runs=500)

In [17]:
monte_carlo_data = np.empty((NRUN,3))
for i in range(NRUN):
    t = t_runs[i]
    monte_carlo_data[i] = decay_fit().values[0:3]

In [18]:
params = np.mean(monte_carlo_data,axis=0)
errors_std= np.std(monte_carlo_data,axis=0)
error_mean = errors_std/np.sqrt(NRUN)

print(f'Parameters Mean Value | Tau : {params[0]}, V: {params[1]}, dm: {params[2]}')
print(f'Standard Deviation | Tau : {errors_std[0]}, V: {errors_std[1]}, dm: {errors_std[2]}')
print(f'Error on the mean | Tau : {error_mean[0]}, V: {error_mean[1]}, dm: {error_mean[2]}')

Parameters Mean Value | Tau : 1.5017369541018022, V: 0.08923535031280681, dm: 19.99714028110935
Standard Deviation | Tau : 0.015481888711368214, V: 0.01431480098986333, dm: 0.07618733205008273
Error on the mean | Tau : 0.0006923711115741189, V: 0.0006401773619543137, dm: 0.0034072010697666676


**Note :** The statistical precision is the error on the mean obtained statistically from the Monte-Carlo.

For the run with 10,000 events, the parameter estimation from minuit is,

| Name  | Value | Error of the Mean(Statistical Precision) |  Standard Deviation
| ---   | ---   | ---   | --
| tau   | 1.5035  | 0.0007  |  0.015     
|  V    | 0.09732 | 0.0006  |   0.01
|  Δm    | 19.993  | 0.003 |  0.07

## Part 2

**2.1 Similarily, now for N = 1e+4 and f = 0.01**

In [20]:
t_runs = decay_toy(f=0.01,mc_runs=500)
monte_carlo_data_f1 = np.empty((NRUN,3))
for i in range(NRUN):
    t = t_runs[i]
    monte_carlo_data_f1[i] = decay_fit().values[0:3]
    
params_f1 = np.mean(monte_carlo_data_f1,axis=0)
errors_f1 = np.std(monte_carlo_data_f1,axis=0)


In [21]:
params_f1 = np.mean(monte_carlo_data_f1,axis=0)
errors_std_f1 = np.std(monte_carlo_data_f1,axis=0)
error_mean_f1 = errors_std_f1/np.sqrt(NRUN)

print(f'Parameters Mean Value | Tau : {params_f1[0]}, V: {params_f1[1]}, dm: {params_f1[2]}')
print(f'Standard Deviation | Tau : {errors_std_f1[0]}, V: {errors_std_f1[1]}, dm: {errors_std_f1[2]}')
print(f'Error on the mean | Tau : {error_mean_f1[0]}, V: {error_mean_f1[1]}, dm: {error_mean_f1[2]}')

Parameters Mean Value | Tau : 1.503187480381766, V: 0.0925512577420372, dm: 20.00064024262163
Standard Deviation | Tau : 0.015043419706015642, V: 0.013656959032409596, dm: 0.07254887523064087
Error on the mean | Tau : 0.0006727621815342175, V: 0.0006107577752479522, dm: 0.003244484334137274


**2.2 Similarily, now for N = 1e+4 and f = 0.03**

In [24]:
t_runs = decay_toy(f=0.03,mc_runs=500)
monte_carlo_data_f3 = np.empty((NRUN,3))
for i in range(NRUN):
    t = t_runs[i]

    monte_carlo_data_f3[i] = decay_fit().values[0:3]
    
params_f3= np.mean(monte_carlo_data_f3,axis=0)
errors_f3 = np.std(monte_carlo_data_f3,axis=0)

/var/folders/2f/y14hzqx53r56d9npnh5jl95m0000gp/T/ipykernel_20132/3094250682.py:97: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  n = integrate.quad(p,XMIN,XMAX)


In [25]:
params_f3 = np.mean(monte_carlo_data_f3,axis=0)
errors_std_f3 = np.std(monte_carlo_data_f3,axis=0)
error_mean_f3 = errors_std_f3/np.sqrt(NRUN)

print(f'Parameters Mean Value | Tau : {params_f3[0]}, V: {params_f3[1]}, dm: {params_f3[2]}')
print(f'Standard Deviation | Tau : {errors_std_f3[0]}, V: {errors_std_f3[1]}, dm: {errors_std_f3[2]}')
print(f'Error on the mean | Tau : {error_mean_f3[0]}, V: {error_mean_f3[1]}, dm: {error_mean_f3[2]}')

Parameters Mean Value | Tau : 1.5008538578655828, V: 0.04214962664048299, dm: 19.901257973748386
Standard Deviation | Tau : 0.015678363363084678, V: 0.013814453474738572, dm: 1.1314632583580222
Error on the mean | Tau : 0.0007011577251159911, V: 0.0006178011408304724, dm: 0.05060057519463889


**2.3 Calculating the shift in the parameters due to non-zero f introduced**

In [26]:
bias_f1 = np.subtract(params,params_f1)
bias_f3 = np.subtract(params,params_f3)

print("Shift in parameters due to resolution error")
print(f'Systematic Shift f= 0.01 | Tau : {bias_f1[0]}, V: {bias_f1[1]}, dm: {bias_f1[2]}')
print(f'Systematic Shift f= 0.03 | Tau : {bias_f3[0]}, V: {bias_f3[1]}, dm: {bias_f3[2]}')
print(f'Error on the mean | Tau : {error_mean[0]}, V: {error_mean[1]}, dm: {error_mean[2]}')


Shift in parameters due to resolution error
Systematic Shift f= 0.01 | Tau : -0.001450526279963782, V: -0.0033159074292303803, dm: -0.0034999615122792704
Systematic Shift f= 0.03 | Tau : 0.000883096236219405, V: 0.047085723672323825, dm: 0.09588230736096293
Error on the mean | Tau : 0.0006923711115741189, V: 0.0006401773619543137, dm: 0.0034072010697666676


| Name  | Bias f = 0.01 | Bias f = 0.03 | Expected Statistical Precision f=0 |
| ---   | ---           |   ---         | ---                             
| tau   | 0.00007         |      0.003   |    0.0002              
|  V    | 0.006        |       0.05    |   0.0002
|  Δm   |  0.002          |       0.06    |  0.001

Bias introduced to the parameters is significantly higher than the expected statistical precision excpet in the case of "Tau" for f = 0.01, where the shift is within the expected statistical precision. This implies there is no bias for tau at f = 0.01. For "V" at f = 0.03, the bias introduced is two order of magnitude higher than the expected statistical precision. For the remaning, the bias is one order of magnitude higher except for  Δm shift being 2x higher for f = 0.01 than the expected statistical precision. The highest shift introduced was for  Δm  at f =0.03 among the other shifts.

## Part 3


**3.1 Generate an array of 500, random s values from a gaussian distribution with $\sigma$ = 0.03 and $\mu$ = 0. Using 500 toy experiments corresponding to each s value.**

In [27]:
Gs = np.random.normal(s_,0.03,NRUN)
t_runs = decay_toy(s=Gs,mc_runs=500)

**3.2 Minuit fit for each s value and recording the shift in the parameters.**

In [28]:

Esys = np.empty((NRUN,4))
for i in range(NRUN):
    t = t_runs[i]
    v = decay_fit().values[0:]
    v_a = decay_fit(s=Gs[i]).values[0:]

    Esys[i] = np.subtract(v_a,v)


/var/folders/2f/y14hzqx53r56d9npnh5jl95m0000gp/T/ipykernel_20132/3094250682.py:97: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  n = integrate.quad(p,XMIN,XMAX)


In [ ]:
Esys_std = np.std(Esys, axis = 0)
print(f'Systematic Error of the Parameters due to Time Acceptance | Tau : {Esys_std[0]}, V: {Esys_std[1]}, dm: {Esys_std[2]}')

Systematic Error of the Parameters due to Time Acceptance | Tau : 0.06559882129801384, V: 0.00019204511872314062, dm: 0.00017823964395478117


| Name  | $E_{sys}$ |  Expected Statistical Precision from before |
| ---   | ---             | ---                             
| tau   | 0.0659         |    0.0002              
|  V    | 0.0002          |   0.0002
|  Δm   |  0.0002           |  0.001

The Systematic errors for each parameter introduced due to Time Acceptance is listed in the table above in the column $E_{sys}$. The systematic error in Δm is within the statistical precision. The systematic error is the most significant for tau, and has higher effect on the total error than the statistical error. For V, the systematic error and the statistical precision are comparable to each other and therefore the systematic error is significant to the total error.